In [16]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, MeanShift, Birch
from scipy.stats import skew
from utils import computeBoundaries, plot_and_segment_cube_signals


In [21]:
def save_feat(material, cube_numbers, base_path):
    
    # Initialize the dictionary to hold the segmented data tensors
    segmented_data_dict = {}

    for cube_number in cube_numbers:
        print(f'Working on cube {cube_number}')
        plot_and_segment_cube_signals(base_path, params_dict, cube_number, segmented_data_dict, threshold=0.1, plot_signals=False)
        print('---' * 10)

    def extract_features(data):
        features = {}

        start_index = int(data.shape[0] * 0.0)
        end_index = int(data.shape[0] * 1.0)

        data_emission = np.mean(data[start_index:end_index, 0, :], axis=0)
        features['mean_emission'] = np.mean(data_emission, axis=0)
        features['std_emission'] = np.std(data_emission, axis=0)
        features['median_emission'] = np.median(data_emission, axis=0)
        features['95th_percentile_emission'] = np.percentile(data_emission, 95, axis=0)
        features['5th_percentile_emission'] = np.percentile(data_emission, 5, axis=0)
        features['skewness_emission'] = skew(data_emission, axis=0)

        data_reflection = np.mean(data[start_index:end_index, 1, :], axis=0)
        features['mean_reflection'] = np.mean(data_reflection, axis=0)
        features['std_reflection'] = np.std(data_reflection, axis=0)
        features['median_reflection'] = np.median(data_reflection, axis=0)
        features['95th_percentile_reflection'] = np.percentile(data_reflection, 95, axis=0)
        features['5th_percentile_reflection'] = np.percentile(data_reflection, 5, axis=0)
        features['skewness_reflection'] = skew(data_reflection, axis=0)

        return features

    features_list = []
    label_list = []
    powers = []
    speeds = []

    for key, value in segmented_data_dict.items():
        power, speed, _ = key
        extracted_features = extract_features(value)
        feats = [v for k, v in extracted_features.items()]

        features_list.append(np.reshape(feats, (1, len(feats))))
        label_list.append((power, speed))
        powers.append(power)
        speeds.append(speed)

    all_data = []
    for single_power in set(powers):
        single_power_data = { (speed, power): val for (power, speed), val in zip(label_list, features_list) if power == single_power }
        if single_power_data:
            all_data.append(single_power_data)


    def prepare_data_for_clustering(data, max_speeds, max_powers):
        to_be_clustered = []
        for k, v in data.items():
            speed, power = k
            num_rows = v.shape[0]

            speed_column = np.full((num_rows, 1), speed / max_speeds)
            power_column = np.full((num_rows, 1), power / max_powers)

            extended_v = np.hstack((v, speed_column, power_column))
            to_be_clustered.append(extended_v)

        X = np.vstack(to_be_clustered)
        X = X[X[:, -2].argsort()]

        return X


    max_speeds = max([max(k[0] for k in data.keys()) for data in all_data])
    max_powers = max([max(k[1] for k in data.keys()) for data in all_data])


    data_dict = {}
    for i in range(len(all_data)):
        data = all_data[i]
        for k, v in data.items():
            speed, power = k
        X = prepare_data_for_clustering(data, max_speeds, max_powers)
        data_dict[power] = X
        
    import os
    import pickle
        
    def load_dictionary(filename):
        with open(filename, 'rb') as file:
            dictionary = pickle.load(file)
        return dictionary

    GT = load_dictionary('./GT')
    
    GT_dict = extract_info_by_material_and_cubes(material, cube_numbers, GT)
    
    # Map modes to binary labels
    mode_mapping = {'T': 1, 'TK': 1, 'K': 1, 'C': 0}
    gt_labels = [mode_mapping[mode] for mode in GT_dict['Modes']]
    gt_speeds = GT_dict['Speeds']
    gt_powers = GT_dict['Power']
    
    local_GT = {}
    local_GT['Labels'] = gt_labels
    local_GT['Speeds'] = gt_speeds
    local_GT['Powers'] = gt_powers
  
    data_dict['GT'] = local_GT


    file_name = material+'_feat.pkl'
    file_path = os.path.join(base_path, file_name)


    with open(file_path, 'wb') as file:
        pickle.dump(data_dict, file)
        

In [22]:
excel_path='./experiment_parameters.xlsx'

# Load parameters from Excel into a dictionary
xl = pd.ExcelFile(excel_path)
params_dict = {sheet_name: xl.parse(sheet_name)[['Speed (mm/s)', 'Power (W)', 'Power perc (%)']].to_dict('records') for sheet_name in xl.sheet_names}


In [23]:
material = '316L'
cube_numbers = [3, 4]

base_path = './Data/Neuchatel_'+material+'/'
save_feat(material, cube_numbers, base_path)

Working on cube 3
Condition 1: Found 10 segments, Avg. Size: 1282.50, Std. Dev: 9.27
Segment Max Size: 1288, Segment Min Size: 1255
Condition 2: Found 10 segments, Avg. Size: 1536.60, Std. Dev: 2.06
Segment Max Size: 1539, Segment Min Size: 1532
Condition 3: Found 10 segments, Avg. Size: 1899.90, Std. Dev: 1.97
Segment Max Size: 1905, Segment Min Size: 1897
Condition 4: Found 10 segments, Avg. Size: 6664.00, Std. Dev: 1.26
Segment Max Size: 6667, Segment Min Size: 6663
Condition 5: Found 10 segments, Avg. Size: 3632.30, Std. Dev: 0.78
Segment Max Size: 3634, Segment Min Size: 3631
Condition 6: Found 10 segments, Avg. Size: 1903.00, Std. Dev: 1.34
Segment Max Size: 1905, Segment Min Size: 1901
Condition 7: Found 10 segments, Avg. Size: 1109.80, Std. Dev: 0.98
Segment Max Size: 1111, Segment Min Size: 1108
Condition 8: Found 10 segments, Avg. Size: 6662.30, Std. Dev: 1.62
Segment Max Size: 6666, Segment Min Size: 6659
Condition 9: Found 10 segments, Avg. Size: 2488.60, Std. Dev: 10.09
Se

In [24]:
material = 'Ti64'
cube_numbers = [3, 4]

base_path = './Data/Neuchatel_'+material+'/'
save_feat(material, cube_numbers, base_path)

Working on cube 3
Condition 1: Found 10 segments, Avg. Size: 1284.40, Std. Dev: 8.82
Segment Max Size: 1288, Segment Min Size: 1258
Condition 2: Found 10 segments, Avg. Size: 1536.70, Std. Dev: 1.10
Segment Max Size: 1538, Segment Min Size: 1535
Condition 3: Found 10 segments, Avg. Size: 1901.20, Std. Dev: 2.23
Segment Max Size: 1903, Segment Min Size: 1895
Condition 4: Found 10 segments, Avg. Size: 6663.80, Std. Dev: 1.08
Segment Max Size: 6666, Segment Min Size: 6662
Condition 5: Found 10 segments, Avg. Size: 3634.20, Std. Dev: 0.75
Segment Max Size: 3635, Segment Min Size: 3633
Condition 6: Found 10 segments, Avg. Size: 1904.40, Std. Dev: 0.66
Segment Max Size: 1905, Segment Min Size: 1903
Condition 7: Found 10 segments, Avg. Size: 1110.80, Std. Dev: 0.87
Segment Max Size: 1112, Segment Min Size: 1109
Condition 8: Found 10 segments, Avg. Size: 6664.20, Std. Dev: 0.87
Segment Max Size: 6666, Segment Min Size: 6663
Condition 9: Found 10 segments, Avg. Size: 2497.80, Std. Dev: 1.33
Seg

In [15]:
local_GT

{'Labels': [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 'Speeds': [1550,
  1300,
  1050,
  300,
  550,
  1050,
  1800,
  300,
  800,
  1300,
  800,
  800,
  1050,
  1550,
  550,
  300,
  1800,
  1300,
  1550,
  550],
 'Powers': [90,
  120,
  90,
  120,
  120,
  120,
  105,
  105,
  90,
  90,
  105,
  120,
  105,
  105,
  105,
  90,
  120,
  105,
  120,
  90]}

In [13]:
gt_powers

[90,
 120,
 90,
 120,
 120,
 120,
 105,
 105,
 90,
 90,
 105,
 120,
 105,
 105,
 105,
 90,
 120,
 105,
 120,
 90]